# Week5 - Ejercicios para practicar

A continuación encontraréis un conjunto de problemas que os pueden servir para practicar los conceptos explicados en esta unidad. Os recomendamos que intentéis realizar estos problemas vosotros mismos y que, una vez realizados, comparéis la solución que proponemos con vuestra solución. No dudéis en dirigir todas las dudas que surjan de la resolución de estos ejercicios, o de las soluciones propuestas, al foro del aula.

1. Cread un código que permita monitorizar el consumo de memoria RAM de la máquina en la que se ejecute. El código guardará los datos de la memoria total y utilizada del sistema durante un periodo de tiempo, capturando los datos en intervalos periódicos.

Estos datos se guardarán en archivos de texto, utilizando un fichero para los datos capturados en cada momento. Así, dentro de la carpeta de datos, habrá una carpeta para los datos de cada día (que tendrá por nombre el año, el mes y el día, escritos seguidos, por ejemplo, `20200318`). Dentro de la carpeta de cada día, habrá un archivo para cada instante de tiempo en el que se hayan obtenido datos (que tendrá por nombre la hora, el minuto y el segundo, separados por guiones bajos, por ejemplo, `14_45_55`). El contenido del archivo serán los dos valores (memoria total y utilizada) separados por comas (por ejemplo, `15571, 4242`).

Cread también el código que permita recuperar todos los datos almacenados y obtener una descripción estadística básica (media, mediana y desviación estándar).

Para ello, implementaremos una serie de funciones que se detallan a continuación.

1.1. Cread una función que reciba como parámetro el nombre de una carpeta (que será `mem_data` por defecto) y cree las carpetas necesarias para almacenar datos para el día actual. Es decir, el código deberá crear, si no existe ya, una carpeta de datos con el nombre que ha recibido como parámetro (o usar `mem_data` si no se ha especificado ningún nombre), y otra carpeta dentro de esta que tenga por nombre el día actual (en el formato año de 4 cifras, mes de 2 cifras, día de 2 cifras, seguidos sin separadores, por ejemplo, `20200318`).

In [12]:
# Importo las librerias necesarias
import os, psutil, datetime, time, re, tarfile

In [13]:
# Respuesta
def create_folder(folderName='mem_data'):
    try:
        # Para saber la ruta absoluta desde .py
        actualFolder = os.path.dirname(__file__)
    except:
        # Para saber el directorio absoluto desde .ipynb
        actualFolder = os.path.abspath('')
    folder = os.path.join(actualFolder, folderName)
    def create_sub_folder():
        today = datetime.date.today().strftime('%Y%m%d')
        subFolderName = os.path.join(folder, today)
        if os.path.isdir(subFolderName) == False:
            os.mkdir(subFolderName)
        return subFolderName
    try:        
        if os.path.isdir(folder) == False:
            os.mkdir(folder)
            subFolderName = create_sub_folder()
        else:
            subFolderName = create_sub_folder()
        return subFolderName
    except:
        return False

1.2. Implementad una función que reciba como parámetro el *path* con la carpeta de la fecha actual (que se ha creado en el apartado anterior) y escriba un fichero con los datos de consumo de memoria del sistema actuales. El archivo debe tener por nombre la hora actual (en el formato `hora_minuto_segundo`, con los ítems separados por guiones bajos, por ejemplo,` 14_45_55`). El contenido del archivo serán los dos valores (memoria total y utilizada) en megabytes separados por comas (por ejemplo, `15571, 4242`).

Para obtener los datos del consumo de memoria, recordad que podéis ejecutar comandos del sistema con el módulo `subprocess` (seguramente necesitaréis buscar información sobre cómo obtener estos datos con comandos de *unix*).

In [14]:
# Respuesta
def create_file(folder):
    if folder == False: return False
    now = datetime.datetime.now().strftime('%H_%M_%S')
    fileName = str(now) + '.log'
    fileName = os.path.join(folder, fileName)
    # Recupero las informaciones de la memoria
    memUso = int(psutil.virtual_memory()[3]/1024**2) # memoria en uso en MB
    memTot = int(psutil.virtual_memory()[0]/1024**2) # memoria total en MB
    # Creo el fichero
    try:
        with open(fileName, 'w') as f:
            f.write(f'{memUso}, {memTot}')
    except:
        return False

1.3. Implementad una función que reciba como parámetros el número de muestras que capturar y el intervalo de tiempo entre cada una de las muestras (en segundos), y que capture los datos del consumo de memoria tantas veces como se haya especificado, esperando el tiempo indicado entre capturas. La función hará uso de las dos funciones definidas anteriormente.


In [15]:
# Respuesta
def capture(interval, n_shot):
    print('Start...')
    count = 0
    intervalTime = time.time()
    while count < n_shot:
        nowTime = time.time()
        if nowTime >= intervalTime:
            result = create_file(create_folder())
            if result != False:
                count += 1
                intervalTime = nowTime + interval
        time.sleep(0.05)
    print('Finish!')

1.4. Llamad a la función definida en el apartado 1.3 y capturad 20 muestras de consumo de memoria, utilizando un intervalo de 3 segundos entre cada captura.

In [16]:
# Respuesta
# capture(3, 20)

1.5. Implementad una función que lea todos los datos que se han capturado, almacenados en una carpeta que recibirá como parámetro (y que, de nuevo, tomará como valor por defecto `mem_data`), y que muestre los siguientes datos:
* El número de muestras leídas.
* La media de la memoria total y utilizada.
* La mediana de la memoria total y utilizada.
* La desviación estándar de la memoria total y utilizada.
* La fecha y hora de la primera y última capturas de las que tenemos datos.

Llamad a la función anterior para obtener un resumen de los datos capturados.

In [17]:
# Respuesta
def logger_reader(folderName='mem_data'):
    # Declaración variables
    fileExtension = 'log'
    fileFormat = re.compile('.._.._...' + fileExtension)
    memUso = [] # Un array para compilar con los datos de todas las lecturas de la memoria en uso
    memTot = [] # Un array para compilar con los datos de todas las lecturas de la memoria total
    # Variables para almacenar las capturas más antiguas y más recientes
    first = '20501212235959'
    last = '0'

    # Obtengo la ruta absoluta de la carpeta en que estamos trabajando
    try:
        # Para saber la ruta absoluta desde .py
        actualFolder = os.path.dirname(__file__)
    except:
        # Para saber el directorio absoluto desde .ipynb
        actualFolder = os.path.abspath('')
    folder = os.path.join(actualFolder, folderName)
    
    # Compruebo que la carpeta exista
    if os.path.isdir(folder) == False:
        print(f'La carpeta {folder} no existe')
    else:
        # Escaneo todas las carpetas y files en mem_data
        with os.scandir(folder) as dir_list:
            for dir in dir_list:
                # Compruebo si es una carpeta
                if os.path.isdir(dir):
                    # Si es una carpeta, actualizo la variable subFolder con la ruta absoluta de dicha carpeta
                    subFolder = os.path.join(folder, dir)
                    # Escaneo todos los ficheros en las subcarpetas
                    with os.scandir(subFolder) as file_list:
                        for file in file_list:
                            # Compruebo si es un fichero y si respeta el formato de nombre utilizado
                            if os.path.isfile(file) and re.match(fileFormat, file.name):
                                # Mantengo en 2 variables la hora más antigua y la más reciente
                                h = dir.name + file.name[:-4].replace('_', '')
                                if h < first: first = h
                                if h > last: last = h
                                # Abro todos los ficheros validos y saco las informaciones
                                with open(file, 'r') as f:
                                    for line in f:
                                        memUso.append(int(line.split(', ')[0]))
                                        memTot.append(int(line.split(', ')[-1]))
        
        # Ahora que tenemos todos los datos, podemos sacar estadísticas
        # El número de muestras leídas.
        print(f'Número de muestras leídas: {len(memUso)}')

        # La media de la memoria total y utilizada.
        medTot = int(sum(memTot)/len(memTot))
        print(f'La media de la memoria total es: {medTot} MB')
        medUso = int(sum(memUso)/len(memUso))
        print(f'La media de la memoria utilizada es: {medUso} MB')
        
        # La mediana de la memoria total y utilizada.
        def mediana(arr):
            arr.sort()
            if len(arr) % 2 == 1:
                return arr[int(len(arr)/2)]
            else:
                a = arr[int((len(arr)/2)-1)]
                p = arr[int(len(arr)/2)]
                return ((a + p)/2)
        print(f'La mediana de la memoria total es {int(mediana(memTot))} MB')
        print(f'La mediana de la memoria utilizada es {int(mediana(memUso))} MB')

        # La desviación estándar de la memoria total y utilizada.
        def st_dev(arr):
            media = sum(arr)/len(arr)
            suma_cuads = 0
            for index in range(len(arr)):
                resta = arr[index] - media
                cuad = resta ** 2
                suma_cuads += cuad
            return int(round((suma_cuads / (len(arr))) ** 0.5, 2))
        print(f'La desviación típica de la memoria total es: {st_dev(memTot)} MB')
        print(f'La desviación típica de la memoria utilizada es: {st_dev(memUso)} MB')

        # La fecha y hora de la primera y última capturas de las que tenemos datos.
        first = datetime.datetime.strptime(first, "%Y%m%d%H%M%S").strftime('%d/%m/%Y a las %H:%M:%S')
        last = datetime.datetime.strptime(last, "%Y%m%d%H%M%S").strftime('%d/%m/%Y a las %H:%M:%S')
        print(f'La primera captura es del {first}')
        print(f'La última captura es del {last}')

logger_reader()

Número de muestras leídas: 100
La media de la memoria total es: 14205 MB
La media de la memoria utilizada es: 10381 MB
La mediana de la memoria total es 14205 MB
La mediana de la memoria utilizada es 10470 MB
La desviación típica de la memoria total es: 0 MB
La desviación típica de la memoria utilizada es: 478 MB
La primera captura es del 22/11/2021 a las 13:47:20
La última captura es del 26/11/2021 a las 10:04:03


1.6. Implementad una función que cree un archivo comprimido con todos los datos almacenados para cada día. La función recibirá como argumento el nombre de la carpeta de datos (por defecto, `mem_data`) y creará tantos ficheros comprimidos como días de los que disponemos datos. Cada archivo comprimido contendrá todos los archivos de datos de ese día.

Llamad a la función anterior y comprobad que se generan los ficheros comprimidos correctamente.


In [18]:
# Respuesta
def compress_folder(folderName='mem_data'):
    ext = '.tar.gz'
    # Obtengo la ruta absoluta de la carpeta en que estamos trabajando
    try:
        # Para saber la ruta absoluta desde .py
        actualFolder = os.path.dirname(__file__)
    except:
        # Para saber el directorio absoluto desde .ipynb
        actualFolder = os.path.abspath('')
    folder = os.path.join(actualFolder, folderName)
    
    # Escaneo la carpeta mem_data y creo un fichero comprimido para cada subcarpeta
    with os.scandir(folder) as dir_list:
        for dir in dir_list:
            if os.path.isdir(dir):
                subFolder = os.path.join(folder, dir.name)
                compressedName = os.path.join(folder, dir.name + ext)
                # Si el fichero no existe creo uno nuevo con el nombre de la subcarpeta
                try:
                    compressedFile = tarfile.open(compressedName, mode = 'x')
                except FileExistsError:
                    print(f'El fichero {dir.name}{ext} ya existe. No se hará ningún cambio.')
                else:
                    with os.scandir(subFolder) as file_list:
                        for file in file_list:
                            if os.path.isfile(file):
                                fileName = os.path.join(subFolder, file.name)
                                compressedFile.add(fileName, file.name)
                    print(f'Compresión del archivo {dir.name}{ext} completada.')
                    compressedFile.close()

compress_folder()                    

Compresión del archivo 20211122.tar.gz completada.
Compresión del archivo 20211123.tar.gz completada.
Compresión del archivo 20211124.tar.gz completada.
Compresión del archivo 20211125.tar.gz completada.
Compresión del archivo 20211126.tar.gz completada.


***

# 8. Bibliografía


## 8.1. Bibliografía básica

La codificación es uno de los detalles importantes que se debe considerar cuando hay que leer y/o escribir un archivo y, a menudo, es el origen de dolores de cabeza en muchos programadores (sobre todo en lenguajes de más bajo nivel que Python). Para entender qué es la codificación de caracteres, conocer cuáles son las codificaciones de caracteres más habituales y saber cómo gestiona Python 3 la codificación, leed ahora la [guía de este enlace](https://realpython.com/python-encodings-guide/#python-3-all-in-on-unicode).


## 8.2. Bibliografía adicional (ampliación de conocimientos)

Esta unidad presenta una introducción a cómo interactuar con el sistema de archivos y, en general, con el sistema operativo, desde Python. Así, como introducción, presenta algunas cuestiones de manera inicial y abre la puerta a explorarlas con más detalle. A continuación se listan algunos enlaces que os servirán para seguir explorando algunos de los temas que trabajamos en la unidad, ya sean puramente de programación en Python como del sistema operativo:

* **El sistema de ficheros de Linux**: en la unidad hablamos de interactuar con el sistema de archivos de Linux, pero no entramos a explicar cómo es este sistema de ficheros. Si deseáis leer una introducción a este sistema, este [*Overview*](https://tldp.org/LDP/intro-linux/html/sect_03_01.html) os puede resultar muy útil.

* **Permisos sobre los ficheros en unix**: si tenéis curiosidad por saber cómo funcionan los bits de permiso de los ficheros en unix, os recomendamos leer las tres partes de la serie de artículos sobre los permisos ([1](http://www.filepermissions.com/articles/what-are-file-permissions-in-linux-and-mac), [2](http://www.filepermissions.com/articles/understanding-octal-file-permissions ) y [3](http://www.filepermissions.com/articles/sticky-bit-suid-and-sgid)).

* **Apertura de archivos desde Python**: la función `open` acepta otros argumentos opcionales que no hemos presentado, y que gestionan detalles como el *buffering* de datos, la codificación, la gestión de los errores, la gestión del salto de línea, etc. El lector interesado puede consultar la [documentación oficial de la función `open`](https://docs.python.org/3/library/functions.html#open) para descubrir cómo funcionan estos argumentos y qué opciones se encuentran disponibles.

* **Compresión de archivos**: existen otros formatos de compresión de datos aparte de los que hemos visto en esta unidad. El lector interesado puede leer la documentación del módulo [`gzip`](https://docs.python.org/3/library/gzip.html) para conocer las funciones que permiten trabajar con archivos gzip desde Python.

* **Lectura de ficheros con pandas**: más allá de los ficheros csv, hay otros formatos que también se utilizan a menudo para intercambiar o guardar datos. Pandas dispone de varias funciones para cargar datos provenientes de los formatos de datos más populares, tales como json ([`read_json`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html#pandas.read_json)) o excel ([`read_excel`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html#pandas.read_excel)).

También os recomendamos revisar la documentación oficial de las funciones y clases descritas en esta unidad, que encontraréis enlazadas en cada uno de los apartados que las describen, con el fin de conocer qué parámetros permiten ajustar su funcionamiento.